# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [5]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy


In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [6]:
print(df['tipo_renda'].value_counts())
print(df['educacao'].value_counts())
print(df['estado_civil'].value_counts())
print(df['tipo_residencia'].value_counts())

Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64
Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64
Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: tipo_residencia, dtype: int64


In [7]:
y, X = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(educacao, Treatment(2)) 
                    + C(estado_civil) 
                    + C(tipo_residencia, Treatment(1)) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

In [8]:
modelo_a = sm.OLS(y,X).fit()

modelo_a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        11:50:47   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.5264      0.219     29.853      0.000       6.098       6.955
C(sexo)[T.M]                                          0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo)[T.True]                           0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel)[T.True]                            0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda)[T.Bolsista]                             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda)[T.Empresário]                           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda)[T.Pensionista]                         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda)[T.Servidor público]                     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]            0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]        0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil)[T.Separado]                           0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil)[T.Solteiro]                           0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil)[T.União]                             -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil)[T.Viúvo]                              0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  


    - As variáveis positivas são: sexo  -> M
                              posse_de_veiculo -> T
                              posse_de_imovel -> T
                              tipo_de_renda -> Empresário/Servidor
                              educacao -> Superior Completo
                              estado_civil -> Separado/Solteiro/Viuvo
                              idade
                              tempo_emprego
                              qt_pessoas_residencia
                              
- A variável qtd_filhos é negativa
- A variável tipo_residencia se mostrou irrelevante.
                           

In [10]:
y_a, X_a = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(educacao, Treatment(2)) 
                    + C(estado_civil) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

In [11]:
modelo_b = sm.OLS(y_a,X_a).fit()

modelo_b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        12:10:17   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            6.5315      0.218     29.894      0.000       6.103       6.960
C(sexo)[T.M]                                         0.7893      0.015     53.999      0.000       0.761       0.818
C(posse_de_veiculo)[T.True]                          0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel)[T.True]                           0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda)[T.Bolsista]                            0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda)[T.Empresário]                          0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda)[T.Pensionista]                        -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda)[T.Servidor público]                    0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment(2))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment(2))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil)[T.Separado]                          0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil)[T.Solteiro]                          0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil)[T.União]                            -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil)[T.Viúvo]                             0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                          -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                        0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     2.13e+

<b> - Removemos a variável tipo_residencia porém o resultado não sofreu grandes alterações.

In [13]:
y_b, X_b = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(estado_civil) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

modelo_c = sm.OLS(y_b,X_b).fit()

modelo_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     453.1
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        12:12:30   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12411   BIC:                         2.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             6.5951      0.219     30.152      0.000       6.166       7.024
C(sexo)[T.M]                          0.7819      0.015     53.480      0.000       0.753       0.811
C(posse_de_veiculo)[T.True]           0.0535      0.014      3.789      0.000       0.026       0.081
C(posse_de_imovel)[T.True]            0.0848      0.014      6.172      0.000       0.058       0.112
C(tipo_renda)[T.Bolsista]             0.2998      0.242      1.241      0.215      -0.174       0.773
C(tipo_renda)[T.Empresário]           0.1655      0.015     11.120      0.000       0.136       0.195
C(tipo_renda)[T.Pensionista]         -0.2540      0.242     -1.051      0.293      -0.728       0.220
C(tipo_renda)[T.Servidor público]     0.0769      0.022      3.472      0.001       0.033       0.120
C(estado_civil)[T.Separado]           0.3241      0.112      2.907      0.004       0.106       0.543
C(estado_civil)[T.Solteiro]           0.2623      0.109      2.403      0.016       0.048       0.476
C(estado_civil)[T.União]             -0.0361      0.025     -1.431      0.153      -0.086       0.013
C(estado_civil)[T.Viúvo]              0.3588      0.116      3.097      0.002       0.132       0.586
qtd_filhos                           -0.2560      0.108     -2.366      0.018      -0.468      -0.044
idade                                 0.0048      0.001      6.147      0.000       0.003       0.006
tempo_emprego                         0.0617      0.001     59.407      0.000       0.060       0.064
qt_pessoas_residencia                 0.2839      0.108      2.629      0.009       0.072       0.496
==============================================================================
Omnibus:                        1.149   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.563   Jarque-Bera (JB):                1.121
Skew:                           0.021   Prob(JB):                        0.571
Kurtosis:                       3.019   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
y_c, X_c = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(estado_civil) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

modelo_d = sm.OLS(y_c,X_c).fit()

modelo_d.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     600.2
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        12:12:56   Log-Likelihood:                -13666.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12415   BIC:                         2.745e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       6.7005      0.220     30.511      0.000       6.270       7.131
C(sexo)[T.M]                    0.7715      0.015     52.631      0.000       0.743       0.800
C(posse_de_veiculo)[T.True]     0.0565      0.014      3.985      0.000       0.029       0.084
C(posse_de_imovel)[T.True]      0.0868      0.014      6.288      0.000       0.060       0.114
C(estado_civil)[T.Separado]     0.3096      0.112      2.763      0.006       0.090       0.529
C(estado_civil)[T.Solteiro]     0.2533      0.110      2.309      0.021       0.038       0.468
C(estado_civil)[T.União]       -0.0281      0.025     -1.110      0.267      -0.078       0.022
C(estado_civil)[T.Viúvo]        0.3317      0.116      2.850      0.004       0.104       0.560
qtd_filhos                     -0.2443      0.109     -2.247      0.025      -0.457      -0.031
idade                           0.0046      0.001      5.783      0.000       0.003       0.006
tempo_emprego                   0.0611      0.001     59.136      0.000       0.059       0.063
qt_pessoas_residencia           0.2677      0.109      2.466      0.014       0.055       0.480
==============================================================================
Omnibus:                        1.331   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.514   Jarque-Bera (JB):                1.311
Skew:                           0.024   Prob(JB):                        0.519
Kurtosis:                       3.013   Cond. No.                     2.13e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
y_d, X_d = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', df)

modelo_e = sm.OLS(y_d,X_d).fit()

modelo_e.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.8
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        12:13:22   Log-Likelihood:                -13672.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       7.2377      0.043    166.875      0.000       7.153       7.323
C(sexo)[T.M]                    0.7694      0.015     52.676      0.000       0.741       0.798
C(posse_de_veiculo)[T.True]     0.0569      0.014      4.022      0.000       0.029       0.085
C(posse_de_imovel)[T.True]      0.0866      0.014      6.275      0.000       0.060       0.114
qtd_filhos                      0.0338      0.019      1.735      0.083      -0.004       0.072
idade                           0.0049      0.001      6.408      0.000       0.003       0.006
tempo_emprego                   0.0610      0.001     59.075      0.000       0.059       0.063
qt_pessoas_residencia          -0.0092      0.016     -0.566      0.572      -0.041       0.023
==============================================================================
Omnibus:                        1.240   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.213
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.019   Cond. No.                         300.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
y_e, X_e = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + idade
                    + tempo_emprego
                    + 1''', df)

modelo_f = sm.OLS(y_e,X_e).fit()

modelo_f.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1098.
Date:                Tue, 05 Jul 2022   Prob (F-statistic):               0.00
Time:                        12:13:41   Log-Likelihood:                -13673.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       7.2223      0.034    213.646      0.000       7.156       7.289
C(sexo)[T.M]                    0.7688      0.015     52.768      0.000       0.740       0.797
C(posse_de_veiculo)[T.True]     0.0561      0.014      3.986      0.000       0.029       0.084
C(posse_de_imovel)[T.True]      0.0866      0.014      6.278      0.000       0.060       0.114
qtd_filhos                      0.0239      0.009      2.767      0.006       0.007       0.041
idade                           0.0049      0.001      6.399      0.000       0.003       0.006
tempo_emprego                   0.0610      0.001     59.084      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.243   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.537   Jarque-Bera (JB):                1.216
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.020   Cond. No.                         223.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- Modelo inicial:

Quantidade de variáveis explicativas: 11 (24 se contarmos os dummies).

R² = 0.357.

R² - ajustado = 0.356.

AIC = 2.719e+04.

 - Modelo refinado:

Quantidade de variáveis explicativas: 6.

R² = 0.347.

R² - ajustado = 0.346.

AIC = 2.736e+04.

- Apesar do R² e R² - ajustado diminuirem e o AIC ter aumentado vemos que essas alterações foram mínimas levando em conta a quantidade de variáveis retiradas do modelo. Então o modelo refinado é muito melhor do que o modelo inicial, ao retirarmos dados que não são estatísticamente relevantes do nosso modelo evitamos overfitting. Pelo princípio da Navalha de Occam, por observarmos que os indicadores sofreram pouca alteração depois das simplificações podemos dizer que o modelo refinado (mais simples) é melhor que o modelo inicial.